# 자연어/놀이터/문서 요약

이 노트북을 차례로 살펴보며 코드의 빈 곳을 채우며 실행하면 자연어 문서 요약 모델의 전반적인 과정을 수행해볼 수 있게 제작되었습니다.

## 과제 설명
법률 문서에 대한 원문을 가장 잘 나타내는 3개의 문장을 추출하는 문서 요약 모델 개발
    
### 데이터 설명
- 입출력: 
    - 입력: 문장별로 나뉜 법률 문서 원문, 예) 법률 문서 = [문장1, 문장2, ..., 문장K], K : 문서 길이
    - 출력: 요약문에 포함될 문장 인덱스 3개
- 데이터셋 구성
    - 학습 데이터:
        - train.json : 24,027개의 법률 문서 아이디 (id), 원문 (article_original), 요약문 (extractive)
    - 테스트 데이터:
        - test.json : 3,004개의 법률 문서 아이디 (id), 원문 (article_original)

### 사용 pretrained 모델
- `beomi/KcELECTRA-base` [Documentation](https://github.com/Beomi/KcELECTRA)
- [BERT모델 논문](https://arxiv.org/pdf/1810.04805.pdf)


## 세팅
### 라이브러리


In [ ]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
# !pip install pytorch-pretrained-bert
# !pip install transformers

In [ ]:
# 필요한 라이브러리 및 코드 파일 불러오기
import os
import time
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

from datetime import datetime, timezone, timedelta
import numpy as np
import random


### 기타
  데이터는 현재 디렉토리의 `data/`폴더 안에 저장합니다.  
- working directory 구조  
  |--code.ipynb  
  |--data/  
  |--|--train/  
  |--|--|--images/  
  |--|--|--|--...  
  |--|--|--labels.json  
  |--|--test/  
  |--|--|--images

- 경로를 설정하세요.

In [ ]:
# 시드 고정
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set device
os.environ["CUDA_VISIBLE_DEVICES"]="1"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 경로 설정
ROOT_PATH = ##### 코드와 데이터가 위치한 경로 지정 #####
DATA_DIR = os.path.join(ROOT_PATH, 'data')
MODEL_DIR = ##### 학습된 모델을 저장할 경로#####


## 데이터 로드
- hyper parameter를 지정하세요.

In [ ]:
# hyper-parameters
TRAIN_BATCH_SIZE = ### 사용자 지정 ###
EVAL_BATCH_SIZE = ### 사용자 지정 ###

# 학습 데이터만 있으니 학습 데이터셋 비율과 validation 데이터셋 비율을 나누기 위해  (원하는 대로 조정 해보세요.)
TRAIN_RATIO = 0.9

#### 코드 채워넣기
- 자연어 데이터를 전처리 하는 `preprocessing`에서 패딩을 실행하는 함수를 만들어보세요.  
  (`##### 코드 #####` 부분)
- `__init__` 에서 tokenizer는 transformers 라이브러리에서 AutoTokenizer를 사용합니다. 이 외에도 원하는 토크나이저를 적용해 다양한 실험을 진행할 수 있습니다.

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertTokenizer
from transformers import AutoTokenizer
from itertools import chain
import json

class CustomDataset(Dataset):
    def __init__(self, data_dir, mode):
        self.data_dir = data_dir
        self.mode = mode
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        self.inputs, self.labels = self.data_loader()

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        
        if os.path.isfile(os.path.join(self.data_dir, self.mode + '_X.pt')):
            inputs = torch.load(os.path.join(self.data_dir, self.mode + '_X.pt'))
            labels = torch.load(os.path.join(self.data_dir, self.mode + '_Y.pt'))

        else:
            file_path = os.path.join(self.data_dir, 'train.json')
            df = pd.read_json(file_path, orient='records', encoding='utf-8-sig')
          
            if self.mode == 'train':
                df = df.loc[:TRAIN_RATIO*int(len(df)), :]
            elif self.mode == 'val':
                df = df.loc[TRAIN_RATIO*int(len(df)):, :]

            inputs = pd.DataFrame(columns=['src'])
            labels = pd.DataFrame(columns=['trg'])
            inputs['src'] =  df['article_original']
            labels['trg'] =  df['extractive']
          
            # Preprocessing
            inputs, labels = self.preprocessing(inputs, labels)
            # Save data
            torch.save(inputs ,os.path.join(self.data_dir, self.mode + '_X.pt'))
            torch.save(labels, os.path.join(self.data_dir, self.mode + '_Y.pt'))

        inputs = inputs.values
        labels = labels.values

        return inputs, labels

    def pad(self, data, pad_id, max_len):
        padded_data = data.map(lambda x : torch.cat([x, torch.tensor([pad_id] * (max_len - len(x)))]))
        return padded_data

    def preprocessing(self, inputs, labels):
        print('Preprocessing ' + self.mode + ' dataset..')
        #Encoding original text
        inputs['src'] = inputs['src'].map(lambda x: torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)),  add_special_tokens=True) for i in range(len(x))]))))
        inputs['clss'] = inputs.src.map(lambda x : torch.cat([torch.where(x == 2)[0], torch.tensor([len(x)])]))
        inputs['segs'] = inputs.clss.map(lambda x : torch.tensor(list(chain.from_iterable([[0] * (x[i+1] - x[i]) if i % 2 == 0 else [1] * (x[i+1] - x[i]) for i, val in enumerate(x[:-1])]))))
        inputs['clss'] = inputs.clss.map(lambda x : x[:-1])

        ##Padding
        max_encoding_len = max(inputs.src.map(lambda x: len(x)))
        max_label_len = max(inputs.clss.map(lambda x: len(x))) 
        
        # mask와 mask clss를 참고하여 0 혹은 -1로 패딩하세요
        inputs['src'] = ##### 코드 #####
        inputs['segs'] = ##### 코드 #####
        inputs['clss'] = ##### 코드 #####
        inputs['mask'] = inputs.src.map(lambda x: ~ (x == 0))
        inputs['mask_clss'] = inputs.clss.map(lambda x: ~ (x == -1))

        #Binarize label {Extracted sentence : 1, Not Extracted sentence : 0}
        labels = labels['trg'].map(lambda  x: torch.tensor([1 if i in x else 0 for i in range(max_label_len)]))

        return inputs, labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return [self.inputs[index][i] for i in range(5)], self.labels[index]


In [ ]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train')
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val')

train_dataloader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False)

## 모델설계
- `##### 코드 #####` 부분을 채워 parameter를 설정해보세요.

In [ ]:
# parameters
EPOCHS = ##### 코드 #####
LEARNING_RATE = ##### 코드 #####
WEIGHT_DECAY = ##### 코드 #####
NUM_WORKERS = ##### 코드 #####
EARLY_STOPPING_PATIENCE = ##### 코드 #####

#### 코드 채워넣기
- 한국어 자연어 처리의 pretrained model인 KcELECTRA 깃헙 [https://github.com/Beomi/KcELECTRA] 을 참고하여 __init__ 부분을 채워보세요. 
  (괄호 안의 `##### 코드 #####` 부분)  
- !주의! 모델이 무거우니 사용하는 파라미터 개수와 개발 환경 등을 고려하여 인코더 등을 선택하세요

In [ ]:
from torch import nn
import transformers
from sklearn.metrics import f1_score

class Summarizer(nn.Module):

    def __init__(self):
        """
        """
        super(Summarizer, self).__init__()
        self.encoder = ##### 코드 #####
        self.fc = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x, segs, clss, mask, mask_clss, sentence_range=None):
        
        top_vec = self.encoder(input_ids = x.long(), attention_mask = mask.float(),  token_type_ids = segs.long()).last_hidden_state
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss.long()]
        sents_vec = sents_vec * mask_clss[:, :, None].float()
        h = self.fc(sents_vec).squeeze(-1)
        sent_scores = self.sigmoid(h) * mask_clss.float()
        return sent_scores
    

In [ ]:
model = Summarizer().to(DEVICE)

In [ ]:
def Hitrate(y_true, y_pred):
    """ Metric 함수 반환하는 함수

    Returns:
        metric_fn (Callable)
    """
    hitrate = np.array([len(list(set(ans).intersection(y_true[i])))/3 for i, ans in enumerate(y_pred)])
    score = np.mean(hitrate)
    return score

In [ ]:
# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = torch.nn.BCELoss(reduction='none')

# Set metrics
metric_fn = Hitrate

In [ ]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int, verbose: bool, logger:logging.RootLogger=None)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.verbose = verbose

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.logger = logger
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단

        Args:
            loss (float):

        Examples:
            
        Note:
            
        """  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopper, Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)
                
        elif loss <= self.min_loss:
            self.save_model = True
            msg = f"Early stopper, Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)


#### 코드 채워넣기
- Train epoch과 Validation epoch 의 loop 안의 `### 코드 ###` 부분을 채워보세요.
- 필요한 부분을 device에 올리는 것도 잊지 마세요.

In [ ]:
class Trainer():
    """ Trainer
        epoch에 대한 학습 및 검증 절차 정의
    
    Attributes:
        model (`model`)
        device (str)
        loss_fn (Callable)
        metric_fn (Callable)
        optimizer (`optimizer`)
        scheduler (`scheduler`)
    """

    def __init__(self, model, device, loss_fn, metric_fn, optimizer=None, scheduler=None, logger=None):
        """ 초기화
        """
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.metric_fn = metric_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.logger = logger

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.train()
        self.train_total_loss = 0
        pred_lst = []
        target_lst = []
        for batch_index, (data, target) in enumerate(dataloader):
            self.optimizer.zero_grad()
            src = data[0].to(self.device)
            clss = ### 코드 ###
            segs = ### 코드 ###
            mask = ### 코드 ###
            mask_clss = ### 코드 ###
            target = target.float().to(self.device)
            sent_score = ### 코드 ### 
            loss = ### 코드 ###
            loss = (loss * mask_clss.float()).sum()
            self.train_total_loss += loss
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
     
            try:
                target_lst.extend(torch.where(target==1)[1].reshape(-1,3).tolist())
            except:
                print(target)
                sys.exit()
                
        self.train_mean_loss = self.train_total_loss / len(dataloader)
        self.train_score = self.metric_fn(y_true=target_lst, y_pred=pred_lst)
        msg = f'Epoch {epoch_index}, Train, loss: {self.train_mean_loss}, Score: {self.train_score}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.eval()
        self.val_total_loss = 0
        pred_lst = []
        target_lst = []

        with torch.no_grad():
            for batch_index, (data, target) in enumerate(dataloader):

                src = data[0].to(self.device)
                clss = ### 코드 ###
                segs = ### 코드 ###
                mask = ### 코드 ###
                mask_clss = ### 코드 ###
                target = target.float().to(self.device)
                sent_score = ### 코드 ###
                loss = ### 코드 ###
                loss = (loss * mask_clss.float()).sum()
                self.val_total_loss += loss
                pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
                target_lst.extend(torch.where(target==1)[1].reshape(-1,3).tolist())
            self.val_mean_loss = self.val_total_loss / len(dataloader)
            self.validation_score = self.metric_fn(y_true=target_lst, y_pred=pred_lst)
            msg = f'Epoch {epoch_index}, Validation, loss: {self.val_mean_loss}, Score: {self.validation_score}'
            print(msg)


In [ ]:
# Set trainer
trainer = Trainer(model, DEVICE, loss_fn, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE, verbose=True)

## 학습
#### 코드 채워넣기
- 위 블록에서 생성한 `train_epoch`과 `valid_epoch`을 사용해 학습을 진행하는 코드를 완성하세요. `##### 코드 #####` 부분 채우기)
- criterion에 맞춰 모델이 자동 업데이트되고, 학습을 마치고 모델을 저장하는 부분 또한 완성하세요. (`### 코드 ###` 부분 채우기)

In [ ]:
# TRAIN
from tqdm import tqdm

start = time.time()
criterion = 0

for epoch_index in tqdm(range(EPOCHS)):
   
    ##### 코드 #####
    
    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if trainer.validation_score > criterion:
        criterion = trainer.validation_score
        check_point:{
            'model' : model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        
        ### 코드 ###
        
print("train finished, best.pt saved.")

## 추론
테스트 데이터의 타겟 변수를 `sample_submission.json` 양식에 맞춰 저장한 파일을 aiconnect 플랫폼을 통해 제출하면 추론 점수를 확인할 수 있습니다.  

"summary_index1", "summary_index2", "summary_index3" 컬럼 값을 여러분의 모델의 추론 결과로 채워 제출 파일을 만듭니다 (현재는 모두 아래 보시는 바와 같이 동일한 값으로 채워져 있습니다). ID 값을 기준으로 채점을 진행하는 점 유의해주시기 바랍니다.

In [ ]:
submit = pd.read_csv(os.path.join(DATA_DIR,'sample_submission.csv'))
submit.head()

#### 코드 채워넣기
- CustomDataset에서처럼 padding의 `### 코드 ###` 부분을 채워 코드를 완성하세요.

In [ ]:
# 테스트 데이터셋 클래스 정의

class TestDataset(Dataset):
    """ CustomDataset과 비슷한 구조이지만 레이블이 주어지지 않음을 염두 """

    def __init__(self, data_dir, mode):
        self.data_dir = data_dir
        self.mode = mode
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        self.inputs = self.data_loader()

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if os.path.isfile(os.path.join(self.data_dir, self.mode+'_X.pt')):
            # torch tensor 불러오기
            inputs = torch.load(os.path.join(self.data_dir, self.mode + '_X.pt'))
        else:
            # json 파일 불러오기
            file_path = os.path.join(self.data_dir, self.mode + '.json')
            df = pd.read_json(file_path, orient='records', encoding='utf-8-sig')
            inputs = pd.DataFrame(columns=['src'])
            inputs['src'] =  df['article_original']
      
            # 전처리
            inputs = self.preprocessing(inputs)
            
            # 다음부터는 전처리 과정을 반복하지 않기 위해 tensor 저장
            torch.save(inputs ,os.path.join(self.data_dir, self.mode + '_X.pt'))

        inputs = inputs.values

        return inputs

    def pad(self, data, pad_id, max_len):
        padded_data = data.map(lambda x : torch.cat([x, torch.tensor([pad_id] * (max_len - len(x)))]))
        return padded_data

    def preprocessing(self, inputs):
        print('Preprocessing ' + self.mode + ' dataset..')
        
        #Encoding original text
        inputs['src'] = inputs['src'].map(lambda x: torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)),  add_special_tokens=True) for i in range(len(x))]))))
        inputs['clss'] = inputs.src.map(lambda x : torch.cat([torch.where(x == 2)[0], torch.tensor([len(x)])]))
        inputs['segs'] = inputs.clss.map(lambda x : torch.tensor(list(chain.from_iterable([[0] * (x[i+1] - x[i]) if i % 2 == 0 else [1] * (x[i+1] - x[i]) for i, val in enumerate(x[:-1])]))))
        inputs['clss'] = inputs.clss.map(lambda x : x[:-1])

        ##Padding
        max_encoding_len = max(inputs.src.map(lambda x: len(x)))
        max_label_len = max(inputs.clss.map(lambda x: len(x)))
        inputs['src'] = self.pad(inputs.src, 0, max_encoding_len)
        inputs['segs'] = ### 코드 ###
        inputs['clss'] = ### 코드 ###
        inputs['mask'] = ### 코드 ###
        inputs['mask_clss'] = inputs.clss.map(lambda x: ~ (x == -1))
     
        return inputs

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return [self.inputs[index][i] for i in range(5)]


In [ ]:
# 테스트 데이터 로드
test_dataset = TestDataset(data_dir=DATA_DIR, mode = 'test')
test_dataloader = DataLoader(dataset=test_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False)

#### 코드 채워넣기
- 학습된 모델을 불러와서 `### 코드 ###` 부분을 채워 추론 코드를 완성하세요
- 필요한 부분을 device에 올리는 것을 잊지 마세요.

In [ ]:
""" 이전에 학습한 모델 weight파일을 불러 추론 """
MODEL_DIR = ### 모델 경로 ###
model = Summarizer().to(DEVICE)
model.load_state_dict(torch.load(MODEL_DIR)['model'])

# 추론
model.eval()

# 추론 결과를 pred 리스트로 저장할 예정
pred_lst = []

with torch.no_grad():
    for batch_index, (data) in enumerate(test_dataloader):

                clss = ### 코드 ###
                segs = ### 코드 ###
                mask = ### 코드 ###
                mask_clss = ### 코드 ###
                sent_score = ### 코드 ###
                pred_lst.extend(torch.topk(sent_score, 3, axis=1).indices.tolist())
 
                src = data[0].to(self.device)

        # 진행과정 출력
        if batch_index % 150 == 0:
            print(f'Prediction: {batch_index}/{len(test_dataloader)} completed')
    print("Prediction all completed")


In [ ]:
print(pred_lst[:5])

#### 코드 채워넣기
- sample submission 양식에 맞춰 제출할 추론 파일을 만들고 저장하세요. (`### 코드 ###` 부분 채우기)

In [ ]:
# 제출 파일 확인
submit[['summary_index1','summary_index2','summary_index3']] = pred_lst
submit.head()

In [ ]:
# 추론 파일 저장
### 코드 ###